**LabTask #4**

**Implement A* Algorithm for Pathfinding**

You are given a 5x5 grid where cells marked as 1 are obstacles, and 0 represents free space.
Your task is to implement the A* algorithm in Python to find the shortest path from the topleft corner (0, 0) to the bottom-right corner (4, 4). The heuristic function should use
Manhattan Distance. Output the grid with the path marked as P.


In [1]:
import heapq  # Importing heapq to use a priority queue for the A* algorithm

# Define the grid (5x5 matrix)
# 0 represents free space, 1 represents obstacles
grid = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
]

# Directions for moving in the grid: (right, down, left, up)
# Each tuple represents a movement in (row, column) format
directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

# Heuristic function using Manhattan distance
# This function estimates the distance from the current node to the goal
def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

# A* algorithm implementation
def a_star_search(grid, start, goal):
    rows, cols = len(grid), len(grid[0])  # Get the number of rows and columns in the grid
    open_list = []  # Priority queue to store nodes that need to be explored
    heapq.heappush(open_list, (0, start))  # Push the start node with f_score = 0
    came_from = {}  # Dictionary to reconstruct the path once the goal is reached
    g_score = {start: 0}  # g_score keeps track of the cost from the start to a given node
    f_score = {start: heuristic(start, goal)}  # f_score = g_score + heuristic

    # Loop until there are nodes to explore
    while open_list:
        current = heapq.heappop(open_list)[1]  # Get the node with the lowest f_score

        # If we reached the goal, reconstruct and return the path
        if current == goal:
            return reconstruct_path(came_from, current)

        # Explore all possible neighbors (up, down, left, right)
        for direction in directions:
            neighbor = (current[0] + direction[0], current[1] + direction[1])

            # Check if the neighbor is within grid bounds and is not an obstacle
            if 0 <= neighbor[0] < rows and 0 <= neighbor[1] < cols and grid[neighbor[0]][neighbor[1]] == 0:
                # Calculate the tentative g_score (distance from start to neighbor)
                tentative_g_score = g_score[current] + 1  # Assuming the cost to move to a neighbor is 1

                # If the neighbor hasn't been explored or we found a shorter path to it
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    came_from[neighbor] = current  # Record the best path to reach this neighbor
                    g_score[neighbor] = tentative_g_score  # Update the g_score of the neighbor
                    f_score[neighbor] = tentative_g_score + heuristic(neighbor, goal)  # Update the f_score
                    heapq.heappush(open_list, (f_score[neighbor], neighbor))  # Add neighbor to the open list

    return None  # Return None if no path was found

# Function to reconstruct the path from the goal to the start
def reconstruct_path(came_from, current):
    path = []  # Initialize an empty path list
    while current in came_from:  # Traverse back from goal to start
        path.append(current)
        current = came_from[current]  # Move to the previous node in the path
    path.append((0, 0))  # Add the start point manually
    return path[::-1]  # Reverse the path to get it from start to goal

# Function to print the grid with the path marked as 'P'
def print_grid_with_path(grid, path):
    for row in range(len(grid)):
        for col in range(len(grid[0])):
            if (row, col) in path:  # If the current cell is part of the path, mark it as 'P'
                print('P', end=' ')
            elif grid[row][col] == 1:  # If it's an obstacle, print '1'
                print('1', end=' ')
            else:  # Otherwise, print '0' for open spaces
                print('0', end=' ')
        print()  # Move to the next row

# Main code to run the A* algorithm and print the grid with the path
start = (0, 0)  # Start position (top-left corner)
goal = (4, 4)  # Goal position (bottom-right corner)
path = a_star_search(grid, start, goal)  # Run the A* search algorithm

# If a path is found, print the grid with the path, otherwise print "No path found"
if path:
    print("Path found:")
    print_grid_with_path(grid, path)
else:
    print("No path found")


Path found:
P 1 0 0 0 
P 1 0 1 0 
P P P 1 0 
1 1 P 1 0 
0 0 P P P 


**Solve the Water-Jug Problem using A* Algorithm**

You are given two jugs with capacities of 4 liters and 3 liters. The goal is to measure exactly
2 liters using these jugs. Implement the A* algorithm to solve the water-jug problem.


In [2]:
import heapq  # Importing heapq to use priority queue for A*

# Define the jug capacities
jug1_capacity = 4  # Jug 1 has a capacity of 4 liters
jug2_capacity = 3  # Jug 2 has a capacity of 3 liters
target = 2  # The goal is to measure exactly 2 liters

# Heuristic function that minimizes the absolute difference to the target
def heuristic(state):
    jug1, jug2 = state
    return abs(jug1 - target) + abs(jug2 - target)

# A* Algorithm implementation for the water jug problem
def a_star_water_jug(start, goal):
    open_list = []  # Priority queue (min-heap) to explore nodes
    heapq.heappush(open_list, (0, start))  # (f_score, (jug1, jug2))
    came_from = {}  # Dictionary to reconstruct the path
    g_score = {start: 0}  # g_score holds the cost to reach each state
    f_score = {start: heuristic(start)}  # f_score = g_score + heuristic

    while open_list:
        current = heapq.heappop(open_list)[1]  # Get the state with the lowest f_score

        # If we reached the goal state, reconstruct the solution path
        if current[0] == goal or current[1] == goal:
            return reconstruct_path(came_from, current)

        # Possible actions: fill, empty, and pour between jugs
        neighbors = generate_neighbors(current)

        for neighbor in neighbors:
            tentative_g_score = g_score[current] + 1  # Each action has a cost of 1

            if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                came_from[neighbor] = current  # Record the best path to this neighbor
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + heuristic(neighbor)  # Update f_score
                heapq.heappush(open_list, (f_score[neighbor], neighbor))  # Add neighbor to the open list

    return None  # Return None if no solution is found

# Generate all possible neighbor states from the current state
def generate_neighbors(state):
    jug1, jug2 = state
    neighbors = []

    # Fill Jug1
    neighbors.append((jug1_capacity, jug2))  # Fill jug1 to full capacity
    # Fill Jug2
    neighbors.append((jug1, jug2_capacity))  # Fill jug2 to full capacity
    # Empty Jug1
    neighbors.append((0, jug2))  # Empty jug1
    # Empty Jug2
    neighbors.append((jug1, 0))  # Empty jug2
    # Pour Jug1 into Jug2
    pour_into_jug2 = min(jug1, jug2_capacity - jug2)  # Pour as much as possible from jug1 to jug2
    neighbors.append((jug1 - pour_into_jug2, jug2 + pour_into_jug2))
    # Pour Jug2 into Jug1
    pour_into_jug1 = min(jug2, jug1_capacity - jug1)  # Pour as much as possible from jug2 to jug1
    neighbors.append((jug1 + pour_into_jug1, jug2 - pour_into_jug1))

    return neighbors

# Reconstruct the path from the goal to the start
def reconstruct_path(came_from, current):
    path = []
    while current in came_from:
        path.append(current)
        current = came_from[current]
    path.append(current)  # Add the start state
    return path[::-1]  # Reverse the path to get it from start to goal

# Main function to run the A* algorithm for the water jug problem
def solve_water_jug_problem():
    start = (0, 0)  # Both jugs start empty
    solution = a_star_water_jug(start, target)

    if solution:
        print("Solution found:")
        for step in solution:
            print(f"Jug1: {step[0]}L, Jug2: {step[1]}L")
    else:
        print("No solution found")

# Run the water jug problem solver
solve_water_jug_problem()


Solution found:
Jug1: 0L, Jug2: 0L
Jug1: 0L, Jug2: 3L
Jug1: 3L, Jug2: 0L
Jug1: 3L, Jug2: 3L
Jug1: 4L, Jug2: 2L


**Implement Hill-Climbing algorithm to solve the 8-queen problem.**

In [3]:
import random

# Define the size of the board (for 8-queen problem)
N = 8

# Function to calculate the number of conflicts for the current board state
def calculate_conflicts(board):
    conflicts = 0
    for i in range(N):
        for j in range(i + 1, N):
            # Check if queens are in the same row or in the same diagonal
            if board[i] == board[j] or abs(board[i] - board[j]) == abs(i - j):
                conflicts += 1
    return conflicts

# Function to generate a random initial board state
def random_board():
    return [random.randint(0, N - 1) for _ in range(N)]

# Hill-Climbing algorithm
def hill_climbing():
    # Start with a random board
    current_board = random_board()
    current_conflicts = calculate_conflicts(current_board)

    while True:
        # Generate all neighbors (move each queen to a different row)
        best_move = None
        best_conflicts = current_conflicts

        for col in range(N):
            for row in range(N):
                if current_board[col] == row:
                    continue
                new_board = current_board[:]
                new_board[col] = row  # Move queen in current column to new row
                new_conflicts = calculate_conflicts(new_board)

                # If this move is better, remember it
                if new_conflicts < best_conflicts:
                    best_conflicts = new_conflicts
                    best_move = new_board

        # If no better move is found, we reached a local minimum, return the result
        if best_conflicts >= current_conflicts:
            break

        # Update the board with the best move
        current_board = best_move
        current_conflicts = best_conflicts

    return current_board, current_conflicts

# Function to print the board
def print_board(board):
    for row in range(N):
        line = ""
        for col in range(N):
            if board[col] == row:
                line += "Q "
            else:
                line += ". "
        print(line)
    print()

# Run the Hill-Climbing algorithm and print the result
solution, conflicts = hill_climbing()
print("Solution found with", conflicts, "conflicts:")
print_board(solution)


Solution found with 2 conflicts:
. . . . . . . Q 
. . Q . . . . . 
. . . . Q . . . 
. Q . . . . . . 
. . . . . . Q . 
Q . . . . . . . 
. . . Q . . . . 
. . . . . Q . . 



**Write a Program to Implement the Mini-Max algorithm for a Tic-Tac-Toe game using python**

In [5]:
# Constants to represent players and the empty spaces
PLAYER_X = 'X'  # Maximizer (AI)
PLAYER_O = 'O'  # Minimizer (Human)
EMPTY = ' '     # Empty cell

# Function to print the Tic-Tac-Toe board
def print_board(board):
    """Prints the current Tic-Tac-Toe board"""
    for row in board:
        print("|".join(row))
        print("-" * 5)

# Function to check if there's a winner
def check_winner(board):
    """Checks if there's a winner on the board and returns the winner"""
    # Check rows for a winner
    for row in board:
        if row[0] == row[1] == row[2] and row[0] != EMPTY:
            return row[0]

    # Check columns for a winner
    for col in range(3):
        if board[0][col] == board[1][col] == board[2][col] and board[0][col] != EMPTY:
            return board[0][col]

    # Check diagonals for a winner
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != EMPTY:
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != EMPTY:
        return board[0][2]

    return None  # No winner yet

# Function to check if the board is full
def is_board_full(board):
    """Checks if the board is full (i.e., no more empty spaces)"""
    for row in board:
        if EMPTY in row:
            return False
    return True

# Mini-Max algorithm
def minimax(board, is_maximizing):
    """
    Mini-Max function to determine the optimal move for the AI (Maximizer).

    Args:
        board: The current Tic-Tac-Toe board.
        is_maximizing: True if the AI (Maximizer) is to move, False if the human (Minimizer) is to move.

    Returns:
        A score indicating the outcome of the game from the perspective of the AI.
    """
    # Check if there's a winner
    winner = check_winner(board)
    if winner == PLAYER_X:
        return 1  # AI wins
    elif winner == PLAYER_O:
        return -1  # Human wins
    elif is_board_full(board):
        return 0  # It's a draw

    # Maximizer's turn (AI - 'X')
    if is_maximizing:
        best_score = -float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_X  # AI makes a move
                    score = minimax(board, False)  # Recur for minimizer
                    board[i][j] = EMPTY  # Undo the move
                    best_score = max(best_score, score)  # Choose the maximum score
        return best_score

    # Minimizer's turn (Human - 'O')
    else:
        best_score = float('inf')
        for i in range(3):
            for j in range(3):
                if board[i][j] == EMPTY:
                    board[i][j] = PLAYER_O  # Human makes a move
                    score = minimax(board, True)  # Recur for maximizer
                    board[i][j] = EMPTY  # Undo the move
                    best_score = min(best_score, score)  # Choose the minimum score
        return best_score

# Function to find the best move for the AI (Maximizer)
def best_move(board):
    """Finds the best possible move for the AI (Maximizer) using the Mini-Max algorithm"""
    best_score = -float('inf')
    move = None
    for i in range(3):
        for j in range(3):
            if board[i][j] == EMPTY:
                board[i][j] = PLAYER_X  # Try the move
                score = minimax(board, False)  # Call minimax for the opponent's turn
                board[i][j] = EMPTY  # Undo the move
                if score > best_score:
                    best_score = score
                    move = (i, j)  # Record the best move
    return move

# Main function to play the Tic-Tac-Toe game
def play_game():
    """Main function to handle the gameplay"""
    # Create an empty 3x3 board
    board = [[EMPTY for _ in range(3)] for _ in range(3)]

    current_player = PLAYER_X  # The AI (Maximizer) starts first

    # Loop until the game ends
    while True:
        print_board(board)  # Print the current board

        # Check if there's a winner or the board is full
        winner = check_winner(board)
        if winner:
            print(f"Player {winner} wins!")
            break
        elif is_board_full(board):
            print("It's a draw!")
            break

        # AI's turn
        if current_player == PLAYER_X:
            print("AI's turn:")
            move = best_move(board)  # Get the best move for the AI
            if move:
                board[move[0]][move[1]] = PLAYER_X  # AI makes the move
            current_player = PLAYER_O  # Switch to the human player

        # Human's turn
        else:
            print("Your turn (Player O):")
            try:
                row = int(input("Enter row (0, 1, 2): "))  # Get the row from the player
                col = int(input("Enter column (0, 1, 2): "))  # Get the column from the player
                if board[row][col] == EMPTY:
                    board[row][col] = PLAYER_O  # Human makes the move
                    current_player = PLAYER_X  # Switch back to the AI
                else:
                    print("Invalid move, try again!")
            except (ValueError, IndexError):
                print("Invalid input, please enter numbers between 0 and 2.")

# Start the game
play_game()


 | | 
-----
 | | 
-----
 | | 
-----
AI's turn:
X| | 
-----
 | | 
-----
 | | 
-----
Your turn (Player O):
Enter row (0, 1, 2): 2
Enter column (0, 1, 2): 0
X| | 
-----
 | | 
-----
O| | 
-----
AI's turn:
X|X| 
-----
 | | 
-----
O| | 
-----
Your turn (Player O):
Enter row (0, 1, 2): 0
Enter column (0, 1, 2): 2
X|X|O
-----
 | | 
-----
O| | 
-----
AI's turn:
X|X|O
-----
 |X| 
-----
O| | 
-----
Your turn (Player O):
Enter row (0, 1, 2): 2
Enter column (0, 1, 2): 1
X|X|O
-----
 |X| 
-----
O|O| 
-----
AI's turn:
X|X|O
-----
 |X| 
-----
O|O|X
-----
Player X wins!
